In [ ]:
#pip install langchain langchain-core langchain-groq groq python-dotenv

In [3]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

In [4]:
# Load environment variables
load_dotenv()

# Initialize Groq chat
groq_chat = ChatGroq(
    temperature=0.7,
    #model_name="llama-3.1-8b-instant",  # Groq's model
    model_name="openai/gpt-oss-120b",
    api_key=os.getenv("GROQ_API_KEY")
)

__The Problem: Why Do We Need Chains?__

Imagine you ask a new, junior comedian to perform:

"Tell me 3 jokes about lawyers that are family-friendly and include a pun in each one."

A junior comedian might:

Tell jokes that aren't about lawyers

Forget the "3 jokes" part and tell just one

Make inappropriate jokes that aren't family-friendly

Forget to include puns

Or ask you after each requirement: "Should I tell another one?", "Was that family-friendly enough?"

Without a chain, an AI model is like that junior comedian. You give it one massive, complex instruction and hope it gets everything right in a single response. This is unreliable and hard to control.

__The Solution: LangChain Chain__

A Chain is a pre-defined sequence of operations for the AI to follow. It's like writing a detailed comedy routine with specific instructions.

Using our example, we would break the complex request down into a chain of simple, reliable steps:

Step 1: Brainstorm lawyer-related topics that are family-friendly

Step 2: Generate puns related to those topics

Step 3: Structure the puns into complete joke formats

Step 4: Ensure exactly 3 jokes are produced

Step 5: Review for family-friendly content

A chain automates this sequence, passing the output from one step as the input to the next.


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

# Define prompt templates (no need for separate Runnable chains)
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a comedian who tells jokes about {topic}."),
        ("human", "Tell me {joke_count} jokes."),
    ]
)

# Create the combined chain using LangChain Expression Language (LCEL)
chain = prompt_template | groq_chat | StrOutputParser()

# Run the chain
result = chain.invoke({"topic": "lawyers", "joke_count": 3})

# Output
print(result)


What the Chain Does Here:

__prompt_template__: The first station. It takes the raw materials (topic and joke_count) and builds a perfectly structured request for the AI.

__groq_chat__: The second station. This is the "AI worker" that does the core task of generating text based on the prompt it receives.

__StrOutputParser()__: The final packaging station. It takes the AI's complex response and neatly extracts just the string of text you want.

Without this chain, you'd have to manually write the code to call each step, pass the data, and handle the output. The chain encapsulates all that logic into a single, reusable, and reliable object.

In [ ]:
#Example 1: Multi-step Reasoning Chain with Memory


from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

# Create memory for conversation history
memory = ConversationBufferMemory(return_messages=True)

# Define multiple prompt templates for different steps
analyze_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful research assistant. Analyze the following topic and break it down into key points."),
    ("human", "Topic: {topic}")
])

expand_prompt = ChatPromptTemplate.from_messages([
    ("system", "Based on the analysis, expand on each key point with detailed explanations and examples."),
    ("human", "Analysis: {analysis}\n\nPlease provide detailed explanations:")
])

summarize_prompt = ChatPromptTemplate.from_messages([
    ("system", "Create a comprehensive summary from the detailed explanations."),
    ("human", "Detailed content: {expanded_content}\n\nCreate a final summary:")
])

# Create the multi-step chain
analysis_chain = analyze_prompt | groq_chat | StrOutputParser()
expansion_chain = expand_prompt | groq_chat | StrOutputParser()
summary_chain = summarize_prompt | groq_chat | StrOutputParser()

# Combine all chains
research_chain = (
    RunnablePassthrough.assign(analysis=analysis_chain)
    | RunnablePassthrough.assign(expanded_content=expansion_chain)
    | summary_chain
)

# Run the chain
result = research_chain.invoke({"topic": "The impact of artificial intelligence on healthcare"})
print("Research Result:")
print(result)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14192\3106303802.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


Research Result:
**Artificial Intelligence in Healthcare: Benefits, Challenges, and Emerging Trends**

Artificial intelligence (AI) has revolutionized the healthcare industry, offering numerous benefits, including improved diagnosis, enhanced patient care, streamlined clinical workflows, personalized medicine, and predictive analytics. However, AI adoption in healthcare also raises concerns about data protection, bias, job displacement, regulatory frameworks, and cybersecurity.

**Benefits:**

AI has improved diagnosis by analyzing large amounts of medical data, identifying patterns, and anomalies that may have gone unnoticed by human clinicians. AI-powered chatbots and virtual assistants provide patients with personalized health advice, medication reminders, and emotional support. AI can automate routine tasks, freeing up healthcare professionals to focus on high-value tasks that require human expertise. AI can also analyze genetic data, medical history, and lifestyle factors to provi

In [ ]:
# Example 2: Conditional Routing Chain with Multiple Outputs

from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch, RunnableLambda

# Define different specialist prompts
technical_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a technical expert. Provide detailed technical explanation about: {query}"),
    ("human", "Please explain this in technical terms:")
])

creative_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a creative writer. Create an engaging story or creative content about: {query}"),
    ("human", "Please create something creative:")
])

business_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a business consultant. Provide strategic business advice about: {query}"),
    ("human", "Please provide business insights:")
])

# Classifier to determine the type of query
classifier_prompt = ChatPromptTemplate.from_messages([
    ("system", "Classify the user's query into one of these categories: technical, creative, business."),
    ("human", "Query: {query}\n\nRespond with only one word: technical, creative, or business")
])

classifier_chain = classifier_prompt | groq_chat | StrOutputParser()

# Create specialized chains
technical_chain = technical_prompt | groq_chat | StrOutputParser()
creative_chain = creative_prompt | groq_chat | StrOutputParser()
business_chain = business_prompt | groq_chat | StrOutputParser()

# Define the routing logic
def route_based_on_type(data):
    query_type = classifier_chain.invoke(data)
    return query_type.strip().lower()

# Create the branching chain
branch_chain = RunnableBranch(
    (lambda x: route_based_on_type(x) == "technical", technical_chain),
    (lambda x: route_based_on_type(x) == "creative", creative_chain),
    (lambda x: route_based_on_type(x) == "business", business_chain),
    creative_chain  # default case
)

# Run the chain with different queries
queries = [
    "Explain how neural networks work",
    "Write a short story about a robot falling in love",
    "What are the market opportunities in renewable energy?"
]

for query in queries:
    print(f"\nQuery: {query}")
    result = branch_chain.invoke({"query": query})
    print(f"Response: {result}\n{'-'*50}")


Query: Explain how neural networks work
Response: **Neural Networks: A Technical Explanation**

Neural networks are a type of machine learning model inspired by the structure and function of the human brain. They consist of interconnected nodes or "neurons" that process and transmit information. In this explanation, we'll delve into the technical details of neural networks, including their architecture, activation functions, and training algorithms.

**Architecture**

A neural network is composed of multiple layers, each containing a number of nodes or "neurons." The layers are typically arranged in a hierarchical structure, with the input layer receiving the initial data, followed by one or more hidden layers, and finally, the output layer producing the predicted results.

The architecture of a neural network can be represented as follows:

* **Input Layer**: This layer receives the input data, which is typically a vector of numbers.
* **Hidden Layers**: These layers contain the node

In [ ]:
# Example 3: Parallel Processing Chain with Aggregation

from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda

# Define different analysis perspectives
sentiment_prompt = ChatPromptTemplate.from_template(
    "Analyze the sentiment of this text and determine if it's positive, negative, or neutral: {text}"
)

key_points_prompt = ChatPromptTemplate.from_template(
    "Extract the 3 most important key points from this text: {text}"
)

summary_prompt = ChatPromptTemplate.from_template(
    "Provide a concise summary of this text (max 100 words): {text}"
)

style_prompt = ChatPromptTemplate.from_template(
    "Analyze the writing style and tone of this text: {text}"
)

# Create parallel chains
sentiment_chain = sentiment_prompt | groq_chat | StrOutputParser()
key_points_chain = key_points_prompt | groq_chat | StrOutputParser()
summary_chain = summary_prompt | groq_chat | StrOutputParser()
style_chain = style_prompt | groq_chat | StrOutputParser()

# Create parallel processing chain
parallel_analysis = RunnableParallel({
    "sentiment": sentiment_chain,
    "key_points": key_points_chain,
    "summary": summary_chain,
    "style_analysis": style_chain
})

# Aggregator chain to combine all analyses
aggregator_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a master analyst. Combine all the following analyses into a comprehensive report.
    
    Sentiment: {sentiment}
    Key Points: {key_points}
    Summary: {summary}
    Style Analysis: {style_analysis}
    
    Create a well-structured final report that incorporates all these insights."""),
    ("human", "Please generate the final comprehensive report:")
])

# Combined chain: parallel analysis -> aggregation
comprehensive_analysis_chain = parallel_analysis | aggregator_prompt | groq_chat | StrOutputParser()

# Sample text for analysis
sample_text = """
The new software update has been received with mixed reactions from users. 
While the improved performance and new features are widely praised, many users 
have reported issues with the updated interface. The dark mode option has been 
particularly popular, but the relocation of key menu items has caused frustration 
among long-time users. Overall, the update represents a significant step forward 
in functionality, though the user experience could benefit from additional refinements.
"""

# Run the comprehensive analysis
print("Running Comprehensive Text Analysis...")
result = comprehensive_analysis_chain.invoke({"text": sample_text})
print("Final Report:")
print(result)

# You can also run individual analyses
print("\n" + "="*60)
print("Individual Analyses:")
print("="*60)

individual_results = parallel_analysis.invoke({"text": sample_text})
for analysis_type, content in individual_results.items():
    print(f"\n{analysis_type.replace('_', ' ').title()}:")
    print(f"{content}\n")

__Key Features Demonstrated:__

Multi-step Chain: Sequential processing with data passing between steps

Conditional Routing: Dynamic routing based on content classification

Parallel Processing: Multiple analyses running simultaneously with aggregation

Memory Integration: Conversation history maintenance

Complex Data Flow: Using RunnablePassthrough and RunnableParallel